# The Stepsize of DSEA+

DSEA+ extends the original DSEA with an adaptively chosen stepsize between iterations. This tutorial assumes you already know the other notebook at `doc/01-getting-started.ipynb`.

In [ ]:
using CherenkovDeconvolution
using ScikitLearn, MLDataUtils, Random
using Discretizers: encode, CategoricalDiscretizer

# load the example data, encode labels with integers
X, y_labels, _ = load_iris()
y = encode(CategoricalDiscretizer(y_labels), y_labels)

# split the data into training and observed data sets
Random.seed!(42) # make split reproducible
(X_train, y_train), (X_data, y_data) = splitobs(shuffleobs((X', y), obsdim = 1), obsdim = 1)

# prepare the arguments for all deconvolution methods
@sk_import naive_bayes : GaussianNB # naive Bayes for DSEA
binning = TreeBinning(3) # up to 3 clusters for IBU & RUN

## Adaptive Step Size

The adaptive step size is specified through the `alpha` argument of DSEA. This argument expects a `CherenkovDeconvolution.Stepsize` object, for which CherenkovDeconvolution.jl provides several implementations.

The most important implementation, `RunStepsize` uses the objective function of the regularized unfolding (RUN) to determine the step size adaptively. We further specify `epsilon`, the minimum Chi square distance between iterations. Convergence is assumed if the distance drops below this threshold.

In this example, convergence is assumed immediately because the input- and output distributions are approximately equal.

In [ ]:
stepsize = RunStepsize(binning; decay=true)
dsea = DSEA(GaussianNB(); K=100, epsilon=1e-6, stepsize=stepsize)
f_dsea = deconvolve(dsea, X_data, X_train, y_train)

## Other Step Sizes

Another adaptive step size, based on a least-square objective, is the `LsqStepsize`. Two decaying step sizes can be obtained with `ExpDecayStepsize` and `MulDecayStepsize`. There is also a `ConstantStepsize`.

If you want to implement additional step size strategies, you only need an implementation of the `stepsize` method for a custom `Stepsize` type.

## Further Documentation

In [ ]:
?Stepsizes.value

In [ ]:
?RunStepsize

In [ ]:
?LsqStepsize

In [ ]:
?ExpDecayStepsize

In [ ]:
?MulDecayStepsize

In [ ]:
?ConstantStepsize

In [ ]:
?DEFAULT_STEPSIZE